# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [1]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
from sklearn.model_selection import train_test_split

A continuación leemos los datasets en dataframes de Pandas.

In [2]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();


df_instancias_aleatorias = pd.read_csv("instancias/aleatorias/indice.csv");
df_instancias_aleatorias_complejidad = pd.read_csv("instancias/aleatorias/complejidad/indice.csv");
df_instancias_tsplib = pd.read_csv("instancias/tsplib/indice.csv");
df_instancias_malo_agm_vec = pd.read_csv("instancias/malo-agm-vec/indice.csv");
df_instancias_aleatorias.head()

,dataset,n,m,archivo
0,aleatorias,10,45,"instancias/aleatorias/RAND-(10, 0).txt"
1,aleatorias,10,45,"instancias/aleatorias/RAND-(10, 1).txt"
2,aleatorias,10,45,"instancias/aleatorias/RAND-(10, 2).txt"
3,aleatorias,10,45,"instancias/aleatorias/RAND-(10, 3).txt"
4,aleatorias,10,45,"instancias/aleatorias/RAND-(10, 4).txt"


La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.
- SEC: Heurística secuencial.
- VEC: Heurística vecindad.
- TABU: Tabú con memoria basada en soluciones exploradas.
- TABU_V: Tabú con memoria basada en estructura (vértices).

In [3]:
def correr_experimento(metodo, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../tp", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise ValueError(F"Hubo un error en la experimentacion para el algoritmo: {metodo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    
    # Leer salida de STDOUT con el costo del camino que encontro
    costo_camino = int(process.stdout.read().split('\n')[0].split(' ')[1]);
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return [tiempo_de_ejecucion, costo_camino];

In [4]:
def correr_experimento_tabu(metodo, k, cant_iteraciones, porcentaje, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../tp", metodo, k, cant_iteraciones, porcentaje], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0:
        raise ValueError(F"Hubo un error en la experimentacion para el algoritmo: {metodo} con la instancia {archivo_instancia}." + metodo_vecindario + ' ' + k + ' ' + cant_iteraciones + process.stderr.read())
    
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    
    # Leer salida de STDOUT con el impacto que obtuvo el algoritmo
    
    costo_camino = int(process.stdout.read().split('\n')[0].split(' ')[1]);
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return [tiempo_de_ejecucion, costo_camino];

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [32]:
experimentos = [];

### Experimento 1, instancias tsplib

Heurística AGM

In [21]:
for i in range(df_instancias_tsplib.shape[0]):
    fila_i = df_instancias_tsplib.iloc[i];
    experimentos.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "AGM", fila_i["archivo"], fila_i["optimo"]]);

Heurística inserción

In [22]:
for i in range(df_instancias_tsplib.shape[0]):
    fila_i = df_instancias_tsplib.iloc[i];
    experimentos.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "INS", fila_i["archivo"], fila_i["optimo"]]);

Heurística vecindad

In [23]:
for i in range(df_instancias_tsplib.shape[0]):
    fila_i = df_instancias_tsplib.iloc[i];
    experimentos.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "VEC", fila_i["archivo"], fila_i["optimo"]]);

Metaheurística tabú search

In [24]:
for i in range(df_instancias_tsplib.shape[0]):
    fila_i = df_instancias_tsplib.iloc[i];
    experimentos.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "TABU_A", fila_i["archivo"], fila_i["optimo"], 3, 7]);
    
for i in range(df_instancias_tsplib.shape[0]):
    fila_i = df_instancias_tsplib.iloc[i];
    experimentos.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "TABU_S", fila_i["archivo"], fila_i["optimo"], 3, 7]);

### Experimento 2, instancias aleatorias

Heurística AGM

In [26]:
for i in range(df_instancias_aleatorias.shape[0]):
    fila_i = df_instancias_aleatorias.iloc[i];
    experimentos.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "AGM", fila_i["archivo"]]);

Heurística vecindad

In [27]:
for i in range(df_instancias_aleatorias.shape[0]):
    fila_i = df_instancias_aleatorias.iloc[i];
    experimentos.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "VEC", fila_i["archivo"]]);

Heurística inserción

In [28]:
for i in range(df_instancias_aleatorias.shape[0]):
    fila_i = df_instancias_aleatorias.iloc[i];
    experimentos.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "INS", fila_i["archivo"]]);

metaheuristica tabu con memoria de aristas

In [29]:
for i in range(df_instancias_aleatorias.shape[0]):
    fila_i = df_instancias_aleatorias.iloc[i];
    experimentos.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "TABU_A", fila_i["archivo"], 3, 5]);

metaheuristica tabu con memoria de soluciones

In [30]:
for i in range(df_instancias_aleatorias.shape[0]):
    fila_i = df_instancias_aleatorias.iloc[i];
    experimentos.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "TABU_S", fila_i["archivo"], 3, 5]);

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [35]:
columnas = ["dataset", "n", "m", "metodo", "obtenido", "optimo", "gap_relativo", "tiempo","k",'iteraciones'];
filas = [];
numero = 1
T = 5
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1

    # Ejecutamos el experimento T veces y obtenemos la mediana.

    tiempos = []
    costos_caminos = []
    for i in range(T):
        if experimento[3][0:4] == 'TABU':#tabu tiene su correr experimentos diferenciado
            print(experimento[5])
            resultados = correr_experimento_tabu(experimento[3], str(experimento[5]), str(experimento[6]), experimento[4]);
            tiempos.append(resultados[0]);
            costos_caminos.append(resultados[1]);
        else:
            resultados = correr_experimento(experimento[3], experimento[4]);
            tiempos.append(resultados[0]);
            costos_caminos.append(resultados[1]);
    tiempo = np.median(tiempos);
    obtenido = np.median(costos_caminos);
    
    if(experimento[0]=="aleatorias"):#si es aleatorio no tenemos los resultados verdaderos
        optimo = '-'
        gap_relativo = '-'
        
    if(experimento[0]=="instancias-tsplib"):#para las instancias de tsplib tenemos los resutados optimos para comparar
        optimo = experimento[5]
        gap_relativo = (obtenido-optimo)/optimo
    
    if experimento[3][0:4] != 'TABU':#si el metodo no es tabu, el k y la cantidad de iteraciones no importan
        iteraciones = '-'
        k = '-'
    else:
        k = experimento[6]
        iteraciones = experimento[6]
    
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], obtenido, optimo, gap_relativo, tiempo, k, iteraciones]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado.csv", index=False, header=True);

'Experimento: 520/520'

3
3
3
3
3


In [ ]:
porcentajes = [0.2, 0.4, 0.6, 0.8, 1]
for i in range(df_instancias_tsplib.shape[0]):
    fila_i = df_instancias_tsplib.iloc[i];
    n = fila_i["n"]
    paso = (n * 10) //100 #10% de n
    paso = max(1, paso)
    for k in range(0, n+1, paso):
        for _iter in range(0, 15):
            for porcentaje in porcentajes:
                tabu_train.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "TABU_A", fila_i["archivo"], fila_i["optimo"], k, _iter, porcentaje]);
                tabu_train.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "TABU_S", fila_i["archivo"], fila_i["optimo"], k, _iter, porcentaje]);

Experimento busqueda de mejores parametros

In [27]:
tabu_train = []

In [28]:
porcentajes = [0.2, 0.4, 0.6, 0.8, 1]
for i in range(df_instancias_tsplib.shape[0] - 1, 0, -1):
    fila_i = df_instancias_tsplib.iloc[i];
    n = fila_i["n"]
    if n == 130:
        continue;
    elif n == 52:
        paso = -4
    elif n == 29:
        paso = -2
    else:
        paso = -1
    for k in range(n, 0, paso):
        for _iter in range(30, 0, -2):
            for porcentaje in porcentajes:
                tabu_train.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "TABU_A", fila_i["archivo"], fila_i["optimo"], k, _iter, porcentaje]);
                tabu_train.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "TABU_S", fila_i["archivo"], fila_i["optimo"], k, _iter, porcentaje]);

In [29]:
columnas = ["dataset", "n", "m", "metodo", "obtenido", "optimo", "gap_relativo", "tiempo","k",'iteraciones', 'porcentaje'];
filas = [];
numero = 1
T = 5
for experimento in tabu_train:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(tabu_train)))
    numero += 1

    # Ejecutamos el experimento T veces y obtenemos la mediana.

    tiempos = []
    costos_caminos = []
    for i in range(T):
        if experimento[3][0:4] == 'TABU':#tabu tiene su correr experimentos diferenciado
            resultados = correr_experimento_tabu(experimento[3], str(experimento[6]), str(experimento[7]), str(experimento[8]), experimento[4]);
            tiempos.append(resultados[0]);
            costos_caminos.append(resultados[1]);
        else:
            resultados = correr_experimento(experimento[3], experimento[4]);
            tiempos.append(resultados[0]);
            costos_caminos.append(resultados[1]);
    tiempo = np.median(tiempos);
    obtenido = np.median(costos_caminos);
    
    if(experimento[0]=="aleatorias" or experimento[0] == "aleatorias-complejidad"):#si es aleatorio no tenemos los resultados verdaderos
        optimo = '-'
        gap_relativo = '-'
        
    if(experimento[0]=="instancias-tsplib"):#para las instancias de tsplib tenemos los resutados optimos para comparar
        optimo = experimento[5]
        gap_relativo = (obtenido-optimo)/optimo
    
    if experimento[3][0:4] != 'TABU':#si el metodo no es tabu, el k y la cantidad de iteraciones no importan
        iteraciones = '-'
        k = '-'
    else:
        k = experimento[6]
        iteraciones = experimento[7]
        porcentaje = experimento[8]
    
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], obtenido, optimo, gap_relativo, tiempo, k, iteraciones, porcentaje]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado_tabu_parametros.csv", index=False, header=True);

'Experimento: 4050/4050'

### Definicion mejores parametros

estamos sumando los gaps relativos para cada cuatro-upla de variables (configuraciones) en base a las cuatro instancias sobre las cuales se corrieron. despues tomamos la que menor suma de gaps tiene, es decir, la que tiene en promedio, el menor gap y asi la mejor performance.

In [7]:
df_resultados_tabu_parametros = pd.read_csv("resultados/resultado_tabu_parametros.csv");
cant_interaciones = range(1,21)
porcentajes = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
metodos = ['TABU_A', 'TABU_S']
ks = range(1, 41)
score_total = np.zeros((len(metodos), len(porcentajes ), len(ks), len(cant_interaciones)))
for pos_metodo in range(len(metodos)):
    metodo_tabu = metodos[pos_metodo]
    for pos_porcentajes in range(len(porcentajes )):
        porcentaje_i = porcentajes[pos_porcentajes]
        for pos_k in range(len(ks)):
            k_i = ks[pos_k]
            for pos_cant_iters in range(len(cant_interaciones)):
                cant_iter = cant_interaciones [pos_cant_iters]
                df_gap_i = df_resultados_tabu_parametros[(df_resultados_tabu_parametros["iteraciones"]== cant_iter) & (df_resultados_tabu_parametros["porcentaje"]==porcentaje_i) & (df_resultados_tabu_parametros["metodo"]==metodo_tabu) & (df_resultados_tabu_parametros["k"]==k_i )].copy();
                gap_i = df_gap_i['gap_relativo'].sum()
                score_total[pos_metodo, pos_porcentajes , pos_k, pos_cant_iters] += gap_i
min_score = 9999
mejores_parametros = []
for pos_metodo in range(len(metodos)):
    for pos_porcentajes in range(len(porcentajes)):
        for pos_k in range(len(ks)):
            for pos_cant_iters in range(len(cant_interaciones)):
                if score_total[pos_metodo, pos_porcentajes , pos_k, pos_cant_iters] < min_score:
                    min_score = score_total[pos_metodo, pos_porcentajes , pos_k, pos_cant_iters]
                    mejores_parametros = [metodos[pos_metodo], porcentajes [pos_porcentajes ], ks[pos_k], cant_interaciones[pos_cant_iters]]

k_optimo = mejores_parametros[2]
iter_optimo = mejores_parametros[3]
porcentaje_optimo = mejores_parametros[1]
display('Mejores parametros impacto: ' + str(mejores_parametros))


"Mejores parametros impacto: ['TABU_S', 0.9, 3, 19]"

Experimento complejidades

In [30]:
experimento_complejidades = []

In [31]:
for i in range(df_instancias_aleatorias_complejidad.shape[0]):
    fila_i = df_instancias_aleatorias_complejidad.iloc[i];
    experimento_complejidades.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "INS", fila_i["archivo"]]);
    experimento_complejidades.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "VEC", fila_i["archivo"]]);
    experimento_complejidades.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "AGM", fila_i["archivo"]]);
    experimento_complejidades.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "TABU_A", fila_i["archivo"], 3, 5]);
    experimento_complejidades.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "TABU_S", fila_i["archivo"], 3, 5]);

In [32]:
columnas = ["dataset", "n", "m", "metodo", "obtenido", "optimo", "gap_relativo", "tiempo","k",'iteraciones'];
filas = [];
numero = 1
T = 5
for experimento in experimento_complejidades:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimento_complejidades)))
    numero += 1

    # Ejecutamos el experimento T veces y obtenemos la mediana.

    tiempos = []
    costos_caminos = []
    for i in range(T):
        if experimento[3][0:4] == 'TABU':#tabu tiene su correr experimentos diferenciado
            resultados = correr_experimento_tabu(experimento[3], str(experimento[5]), str(experimento[6]), experimento[4]);
            tiempos.append(resultados[0]);
            costos_caminos.append(resultados[1]);
        else:
            resultados = correr_experimento(experimento[3], experimento[4]);
            tiempos.append(resultados[0]);
            costos_caminos.append(resultados[1]);
    tiempo = np.median(tiempos);
    obtenido = np.median(costos_caminos);
    
    if(experimento[0]=="aleatorias" or experimento[0] == "aleatorias-complejidad"):#si es aleatorio no tenemos los resultados verdaderos
        optimo = '-'
        gap_relativo = '-'
        
    if(experimento[0]=="instancias-tsplib"):#para las instancias de tsplib tenemos los resutados optimos para comparar
        optimo = experimento[5]
        gap_relativo = (obtenido-optimo)/optimo
    
    if experimento[3][0:4] != 'TABU':#si el metodo no es tabu, el k y la cantidad de iteraciones no importan
        iteraciones = '-'
        k = '-'
    else:
        k = experimento[5]
        iteraciones = experimento[6]
    
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], obtenido, optimo, gap_relativo, tiempo, k, iteraciones]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado_complejidades.csv", index=False, header=True);

'Experimento: 535/535'

In [23]:
experimentos_complejidades_tabu = []

In [24]:
porcentajes_i = range(0, 101)
porcentajes = []
for i in range(0,101):
    porcentajes.append(porcentajes_i[i]/100)
porcentajes[0] += 0.01
iteraciones = range(0, 101)
ks = range(0, 101)
data = df_instancias_tsplib.iloc[2]

In [25]:
for k in ks:
    experimentos_complejidades_tabu.append([data["dataset"], data["n"], data["m"], "TABU_S", data["archivo"], data["optimo"], k, 15, 1]);
for _iter in iteraciones:
    experimentos_complejidades_tabu.append([data["dataset"], data["n"], data["m"], "TABU_S", data["archivo"], data["optimo"], 3, _iter, 0.3]);
for porcentaje in porcentajes:
    experimentos_complejidades_tabu.append([data["dataset"], data["n"], data["m"], "TABU_S", data["archivo"], data["optimo"], 3, 5, porcentaje]);

for k in ks:
    experimentos_complejidades_tabu.append([data["dataset"], data["n"], data["m"], "TABU_A", data["archivo"], data["optimo"], k, 5, 0.3]);
for _iter in iteraciones:
    experimentos_complejidades_tabu.append([data["dataset"], data["n"], data["m"], "TABU_A", data["archivo"], data["optimo"], 3, _iter, 0.3]);
for porcentaje in porcentajes:
    experimentos_complejidades_tabu.append([data["dataset"], data["n"], data["m"], "TABU_A", data["archivo"], data["optimo"], 3, 5, porcentaje]);

In [26]:
columnas = ["dataset", "n", "m", "metodo", "obtenido", "optimo", "gap_relativo", "tiempo","k",'iteraciones', 'porcentaje'];
filas = [];
numero = 1
T = 5
for experimento in experimentos_complejidades_tabu:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos_complejidades_tabu)))
    numero += 1

    # Ejecutamos el experimento T veces y obtenemos la mediana.

    tiempos = []
    costos_caminos = []
    for i in range(T):
        if experimento[3][0:4] == 'TABU':#tabu tiene su correr experimentos diferenciado
            resultados = correr_experimento_tabu(experimento[3], str(experimento[6]), str(experimento[7]), str(experimento[8]), experimento[4]);
            tiempos.append(resultados[0]);
            costos_caminos.append(resultados[1]);
        else:
            resultados = correr_experimento(experimento[3], experimento[4]);
            tiempos.append(resultados[0]);
            costos_caminos.append(resultados[1]);
    tiempo = np.median(tiempos);
    obtenido = np.median(costos_caminos);
    
    if(experimento[0]=="aleatorias" or experimento[0] == "aleatorias-complejidad"):#si es aleatorio no tenemos los resultados verdaderos
        optimo = '-'
        gap_relativo = '-'
        
    if(experimento[0]=="instancias-tsplib"):#para las instancias de tsplib tenemos los resutados optimos para comparar
        optimo = experimento[5]
        gap_relativo = (obtenido-optimo)/optimo
    
    if experimento[3][0:4] != 'TABU':#si el metodo no es tabu, el k y la cantidad de iteraciones no importan
        iteraciones = '-'
        k = '-'
    else:
        k = experimento[6]
        iteraciones = experimento[7]
        porcentaje = experimento[8]
    
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], obtenido, optimo, gap_relativo, tiempo, k, iteraciones, porcentaje]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado_tabu_complejidades.csv", index=False, header=True);

'Experimento: 606/606'

### Experimento 3, instancias donde las heurísticas no dan bien

In [ ]:
experimentos_malos = []

#Heurística VEC
for i in range(df_instancias_malo_agm_vec.shape[0]):
    fila_i = df_instancias_malo_agm_vec.iloc[i];
    experimentos_malos.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "VEC", fila_i["archivo"], fila_i["optimo"], fila_i["esperado"], fila_i["k"]]);

#Heurística AGM
for i in range(df_instancias_malo_agm_vec.shape[0]):
    fila_i = df_instancias_malo_agm_vec.iloc[i];
    experimentos_malos.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "AGM", fila_i["archivo"], fila_i["optimo"], fila_i["esperado"], fila_i["k"]]);


### Experimento casos malos 
columnas = ["dataset", "n", "m", "metodo", "obtenido", "optimo", "esperado", "tiempo","k"];
filas = [];
numero = 1
T = 5
for experimento in experimentos_malos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos_malos)))
    numero += 1

    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    costos_caminos = []
    for i in range(T):
        resultados = correr_experimento(experimento[3], experimento[4]);
        tiempos.append(resultados[0]);
        costos_caminos.append(resultados[1]);
    tiempo = np.median(tiempos);
    obtenido = np.median(costos_caminos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], obtenido, experimento[5], experimento[6], tiempo, experimento[7]]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultados_malos.csv", index=False, header=True);

### Experimentos 5

Corremos a los métodos con nuevas instancias y a tabú con los parámetros óptimos.

In [10]:
experimento5 = []
for i in range(df_instancias_tsplib.shape[0]):
    fila_i = df_instancias_tsplib.iloc[i];
    if (fila_i['n'] > 190 and fila_i['n'] < 320): # rango nuevas instancias
        experimento5.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "VEC", fila_i["archivo"], fila_i["optimo"]]);
        experimento5.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "INS", fila_i["archivo"], fila_i["optimo"]]);
        experimento5.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "AGM", fila_i["archivo"], fila_i["optimo"]]);
        experimento5.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "TABU_A", fila_i["archivo"], fila_i["optimo"], k_optimo, iter_optimo, porcentaje_optimo]);
        experimento5.append([fila_i["dataset"], fila_i["n"], fila_i["m"], "TABU_S", fila_i["archivo"], fila_i["optimo"], k_optimo, iter_optimo, porcentaje_optimo]);
    

In [11]:
columnas = ["dataset", "n", "m", "metodo", "obtenido", "optimo", "gap_relativo", "tiempo","k",'iteraciones', 'porcentaje'];
filas = [];
numero = 1
T = 5
for experimento in experimento5:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimento5)))
    numero += 1

    # Ejecutamos el experimento T veces y obtenemos la mediana.

    tiempos = []
    costos_caminos = []
    for i in range(T):
        if experimento[3][0:4] == 'TABU':#tabu tiene su correr experimentos diferenciado
            resultados = correr_experimento_tabu(experimento[3], str(experimento[6]), str(experimento[7]), str(experimento[8]), experimento[4]);
            tiempos.append(resultados[0]);
            costos_caminos.append(resultados[1]);
        else:
            resultados = correr_experimento(experimento[3], experimento[4]);
            tiempos.append(resultados[0]);
            costos_caminos.append(resultados[1]);
    tiempo = np.median(tiempos);
    obtenido = np.median(costos_caminos);
    
    if(experimento[0]=="aleatorias" or experimento[0] == "aleatorias-complejidad"):#si es aleatorio no tenemos los resultados verdaderos
        optimo = '-'
        gap_relativo = '-'
        
    if(experimento[0]=="instancias-tsplib"):#para las instancias de tsplib tenemos los resutados optimos para comparar
        optimo = experimento[5]
        gap_relativo = (obtenido-optimo)/optimo
    
    if experimento[3][0:4] != 'TABU':#si el metodo no es tabu, el k y la cantidad de iteraciones no importan
        iteraciones = '-'
        k = '-'
        porcentaje = '-'
    else:
        k = experimento[6]
        iteraciones = experimento[7]
        porcentaje = experimento[8]
    
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], obtenido, optimo, gap_relativo, tiempo, k, iteraciones, porcentaje]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado_exp5.csv", index=False, header=True);

'Experimento: 15/15'